In [1]:
!pip install -q ultralytics

In [2]:
import os
from pathlib import Path
import yaml

# For dataset sanity checks
from collections import Counter

# Detect project root (same pattern as other notebooks)
CWD = Path().resolve()
if CWD.name == "notebooks":
    PROJECT_ROOT = CWD.parent
else:
    PROJECT_ROOT = CWD

print("Current working dir:", CWD)
print("Assumed project root:", PROJECT_ROOT)

# Candidate paths for object detection dataset (handle name variations)
candidates = [
    PROJECT_ROOT / "data" / "object_detection_Dataset",
    PROJECT_ROOT / "data" / "object_detection_dataset",
]

DETECTION_DIR = None
for c in candidates:
    if c.exists():
        DETECTION_DIR = c
        break

if DETECTION_DIR is None:
    raise FileNotFoundError(
        f"❌ Could not find object detection dataset in: {candidates}. "
        "Check folder name under data/."
    )

print("Using detection dataset path:", DETECTION_DIR)

CONFIGS_DIR = PROJECT_ROOT / "configs"
DETECTION_MODELS_DIR = PROJECT_ROOT / "models" / "detection"

CONFIGS_DIR.mkdir(parents=True, exist_ok=True)
DETECTION_MODELS_DIR.mkdir(parents=True, exist_ok=True)


Current working dir: F:\Aerial_Object_Classification_Detection\notebooks
Assumed project root: F:\Aerial_Object_Classification_Detection
Using detection dataset path: F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset


In [3]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

def count_yolo_split(split_name):
    images_dir = DETECTION_DIR / split_name / "images"
    labels_dir = DETECTION_DIR / split_name / "labels"
    
    if not images_dir.exists() or not labels_dir.exists():
        print(f"⚠️ Missing folder for split '{split_name}':")
        print("   images:", images_dir, "exists:", images_dir.exists())
        print("   labels:", labels_dir, "exists:", labels_dir.exists())
        return 0, 0
    
    image_files = [f for f in images_dir.iterdir() if f.suffix.lower() in IMAGE_EXTS]
    label_files = [f for f in labels_dir.iterdir() if f.suffix.lower() == ".txt"]
    
    print(f"Split: {split_name}")
    print("  Images:", len(image_files))
    print("  Labels:", len(label_files))
    
    return len(image_files), len(label_files)

train_imgs, train_lbls = count_yolo_split("train")
val_imgs,   val_lbls   = count_yolo_split("valid")  # folder is 'valid' in your data
test_imgs,  test_lbls  = count_yolo_split("test")

if train_imgs == 0 or train_lbls == 0:
    raise ValueError("❌ Train split has zero images or labels. Check dataset placement.")


Split: train
  Images: 2728
  Labels: 2728
Split: valid
  Images: 448
  Labels: 448
Split: test
  Images: 224
  Labels: 224


In [4]:
import random

def inspect_random_label(split_name="train", num_lines=5):
    labels_dir = DETECTION_DIR / split_name / "labels"
    if not labels_dir.exists():
        print(f"Labels dir not found: {labels_dir}")
        return
    
    label_files = [f for f in labels_dir.iterdir() if f.suffix.lower() == ".txt"]
    if not label_files:
        print(f"No label files in {labels_dir}")
        return
    
    sample_file = random.choice(label_files)
    print(f"Sample label file ({split_name}):", sample_file)
    
    with open(sample_file, "r") as f:
        lines = f.read().strip().splitlines()
    
    print("\nContent (up to first 5 lines):")
    for line in lines[:num_lines]:
        print(line)

inspect_random_label("train")


Sample label file (train): F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\train\labels\0d0cb85bcdc20bae_jpg.rf.bd32b8f0b737dda8b87fa16ab14074af.txt

Content (up to first 5 lines):
0 0.5084134615384616 0.5168269230769231 0.8966346153846154 0.8822115384615384


In [5]:
yolo_data = {
    # Note: 'val' key name is fixed, but folder can be called 'valid'
    "train": str((DETECTION_DIR / "train" / "images").resolve()),
    "val":   str((DETECTION_DIR / "valid" / "images").resolve()),
    "test":  str((DETECTION_DIR / "test" / "images").resolve()),
    "nc": 2,
    "names": ["bird", "drone"]
}

yaml_path = CONFIGS_DIR / "yolov8_data.yaml"

with open(yaml_path, "w") as f:
    yaml.dump(yolo_data, f, default_flow_style=False)

print("✅ YOLO data.yaml created at:", yaml_path)
print("\nFile content:")
print(yaml.dump(yolo_data, default_flow_style=False))


✅ YOLO data.yaml created at: F:\Aerial_Object_Classification_Detection\configs\yolov8_data.yaml

File content:
names:
- bird
- drone
nc: 2
test: F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\test\images
train: F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\train\images
val: F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\valid\images



In [7]:
%pip install ultralytics
try:
    from ultralytics import YOLO
except ImportError as e:
    raise ImportError(
        "Ultralytics YOLO is not installed. "
        "Install it with: pip install ultralytics"
    ) from e

print("✅ Ultralytics YOLO imported successfully.")


  Using cached torchvision-0.24.1-cp313-cp313-win_amd64.whl.metadata (5.9 kB)
  Using cached polars-1.35.2-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 9.5 MB/s  0:00:00
Using cached polars-1.35.2-py3-none-any.whl (783 kB)
Using cached torchvision-0.24.1-cp313-cp313-win_amd64.whl (4.3 MB)
Using cached ultralytics_thop-2.0.18-py3-none-any.whl (28 kB)

   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   ---------------------------------------- 0/4 [polars]
   --

In [11]:
# Choose which YOLOv8 model to use (you can change to yolov8s.pt, yolov8m.pt, etc.)
MODEL_NAME = "yolov8n.pt"

# Initialize model (pretrained)
yolo_model = YOLO(MODEL_NAME)

# Training hyperparameters
EPOCHS = 10      # you can start with 30 to test
BATCH_SIZE = 10
IMG_SIZE = 80

results = yolo_model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=str(DETECTION_MODELS_DIR),   # where to save runs
    name="yolov8n_bird_drone",           # run name
    exist_ok=True                         # overwrite if same name exists
)


Ultralytics 8.3.233  Python-3.13.7 torch-2.9.1+cpu CPU (Intel Core i5-10210U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=10, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=F:\Aerial_Object_Classification_Detection\configs\yolov8_data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=80, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_bird_drone, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

In [12]:
# YOLOv8 usually saves to so    mething like:
# models/detection/yolov8n_bird_drone/weights/best.pt

run_dir = DETECTION_MODELS_DIR / "yolov8n_bird_drone"
weights_dir = run_dir / "weights"
best_weights = weights_dir / "best.pt"
last_weights = weights_dir / "last.pt"

print("Run directory:", run_dir)
print("Best weights path:", best_weights, "exists:", best_weights.exists())
print("Last weights path:", last_weights, "exists:", last_weights.exists())


Run directory: F:\Aerial_Object_Classification_Detection\models\detection\yolov8n_bird_drone
Best weights path: F:\Aerial_Object_Classification_Detection\models\detection\yolov8n_bird_drone\weights\best.pt exists: True
Last weights path: F:\Aerial_Object_Classification_Detection\models\detection\yolov8n_bird_drone\weights\last.pt exists: True


In [13]:
# Validation on the val set
val_results = yolo_model.val(
    data=str(yaml_path),
    split="val",        # uses 'val' key from yaml, which points to valid/images
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=str(DETECTION_MODELS_DIR),
    name="yolov8n_bird_drone_val",
    exist_ok=True
)

print("Validation metrics object:", val_results)


WARNING imgsz=[80] must be multiple of max stride 32, updating to [96]
Ultralytics 8.3.233  Python-3.13.7 torch-2.9.1+cpu CPU (Intel Core i5-10210U 1.60GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 69.022.0 MB/s, size: 14.8 KB)
val: Scanning F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\valid\labels.cache... 448 images, 6 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 448/448 459.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 45/45 6.7it/s 6.7s0.1s
                   all        448        663      0.714      0.559      0.591      0.339
                  bird        217        414      0.605      0.352      0.367      0.173
                 drone        225        249      0.823      0.766      0.815      0.506
Speed: 0.1ms preprocess, 8.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to 

In [14]:
# Choose some test images folder
test_images_dir = DETECTION_DIR / "test" / "images"
print("Test images directory:", test_images_dir)

if not test_images_dir.exists():
    raise FileNotFoundError(f"Test images folder not found: {test_images_dir}")

# Run inference using best weights, if available
if best_weights.exists():
    infer_model = YOLO(str(best_weights))
else:
    print("⚠️ best.pt not found, using current model in memory.")
    infer_model = yolo_model

# Limit predictions to a few images to visualize
# You can change 'max_images' if you want more
max_images = 10
image_paths = [str(p) for p in list(test_images_dir.iterdir())[:max_images]]

print(f"Running inference on {len(image_paths)} test images...")

pred_results = infer_model.predict(
    source=image_paths,
    imgsz=IMG_SIZE,
    project=str(DETECTION_MODELS_DIR),
    name="yolov8n_bird_drone_infer",
    exist_ok=True,
    save=True,       # saves images with bounding boxes
    save_txt=False   # set True if you also want YOLO-format txt outputs
)

print("✅ Inference completed. Check output folder:")
print(DETECTION_MODELS_DIR / "yolov8n_bird_drone_infer")


Test images directory: F:\Aerial_Object_Classification_Detection\data\object_detection_Dataset\test\images
Running inference on 10 test images...

WARNING imgsz=[80] must be multiple of max stride 32, updating to [96]
0: 96x96 1 bird, 6.5ms
1: 96x96 1 bird, 6.5ms
2: 96x96 (no detections), 6.5ms
3: 96x96 1 bird, 6.5ms
4: 96x96 1 bird, 6.5ms
5: 96x96 2 birds, 6.5ms
6: 96x96 4 birds, 6.5ms
7: 96x96 1 bird, 6.5ms
8: 96x96 2 birds, 6.5ms
9: 96x96 2 birds, 6.5ms
Speed: 0.5ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 96, 96)
Results saved to F:\Aerial_Object_Classification_Detection\models\detection\yolov8n_bird_drone_infer
✅ Inference completed. Check output folder:
F:\Aerial_Object_Classification_Detection\models\detection\yolov8n_bird_drone_infer
